In [1]:
# !nvidia-smi

# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
# CUDA = format_cuda_version(CUDA_version)

In [2]:
import json
import os
# import uuid  # https://docs.python.org/3/library/uuid.html
# import structlog  # for event logging
# # from dotenv import load_dotenv # enviornment vars if we want

# from pygtail import Pygtail
# import boto3
# from minio import Minio
# from dotenv import load_dotenv
from datetime import timedelta, datetime
from time import sleep
from sys import argv
# import threading
# from smart_open import smart_open
import gensim
import gensim.corpora as corpora

# setup to import the preprocessor
import sys
# from botocore.errorfactory import ClientError
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()
import torch_geometric
from functools import reduce
from tqdm import tqdm

from Preprocessing import Preprocessor
preprocessor = Preprocessor(0)
tqdm.pandas()


c:\Python310\lib\site-packages\past\builtins\misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
c:\Python310\lib\site-packages\joblib\backports.py:7: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  from distutils.version import LooseVersion


In [3]:
joined_exists = os.path.exists('khan_joined.csv')
khan = pd.DataFrame([])
if joined_exists:
    khan = pd.read_csv('khan_joined.csv')
khan.head(5)

,course,unit,lesson,video_title,about,transcript,topic,transcript_cleaned,transcript_n_entries,tokens
0,computer,Intro to JS: Drawing & Animation,Intro to programming,What is Programming?,Programming is the process of creating a set o...,"Hi, welcome to programming! If you've never le...",Computing,"['Angry Birds', 'and of course', 'were giving ...",183,"['hi,', 'welcome', 'programming!', 'never', 'l..."
1,computer,Intro to JS: Drawing & Animation,Coloring,The Power of the Docs,Created by Pamela Fox.,Voiceover: Ok so you've\r\nmade a few programs...,Computing,"['because 1 degree to 270 degrees,\r\nthats wh...",542,"['voiceover:', 'ok', 'made', 'programs,', 'mig..."
2,computer,Intro to HTML/CSS: Making webpages,Further learning,HTML validation,Learn how to validate your webpages with the W...,"- [Voiceover] On Khan Academy, we pop up the o...",Computing,['this is a classic thing to forget is that al...,172,"['-', 'khan', 'academy,', 'pop', 'oh', 'noes',..."
3,computer,Intro to SQL: Querying and managing data,SQL basics,Welcome to SQL,SQL is useful for creating and querying relati...,- [Instructor] The world is full of data. Ever...,Computing,['just by mapping user IDs to badge IDs. Thats...,203,"['-', 'world', 'full', 'data.', 'every', 'app'..."
4,computer,Intro to SQL: Querying and managing data,SQL basics,S-Q-L or SEQUEL?,How is it pronounced? Why? Let's discuss...,"At this point, you've probably heard me\r\npro...",Computing,"['At this point', 'by pronouncing SQL the long...",129,"['point,', 'probably', 'heard', 'pronounce', '..."


In [4]:
if not joined_exists:
    computing = pd.read_csv("Datasets\\KhanAcademy\\Computing.csv")
    computing = computing.dropna()
    computing['topic'] = 'Computing'
    computing.info()

In [5]:
if not joined_exists:
    economics = pd.read_csv("Datasets\\KhanAcademy\\Economics.csv")
    economics = economics.dropna()
    economics['topic'] = 'Economics'
    economics.info()

In [6]:
if not joined_exists:
    humanities = pd.read_csv("Datasets\\KhanAcademy\\Humanities.csv")
    humanities = humanities.dropna()
    humanities['topic'] = 'Humanities'
    humanities.info()

In [7]:
if not joined_exists:
    math = pd.read_csv("Datasets\\KhanAcademy\\Math.csv")
    math = math.dropna()
    math['topic'] = 'Math'
    math.info()

In [8]:
if not joined_exists:
    science = pd.read_csv("Datasets\\KhanAcademy\\Science.csv")
    science = science.dropna()
    science['topic'] = 'Science'
    science.info()

In [9]:
ted_main = pd.read_csv("Datasets\\TEDTalksDataset\\ted_main.csv")
transcripts = pd.read_csv("Datasets\\TEDTalksDataset\\transcripts.csv")
validation = ted_main.join(transcripts, lsuffix='url', rsuffix='url', sort=True)
validation = validation.dropna()
validation.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 0 to 2466
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   comments            2461 non-null   int64 
 1   description         2461 non-null   object
 2   duration            2461 non-null   int64 
 3   event               2461 non-null   object
 4   film_date           2461 non-null   int64 
 5   languages           2461 non-null   int64 
 6   main_speaker        2461 non-null   object
 7   name                2461 non-null   object
 8   num_speaker         2461 non-null   int64 
 9   published_date      2461 non-null   int64 
 10  ratings             2461 non-null   object
 11  related_talks       2461 non-null   object
 12  speaker_occupation  2461 non-null   object
 13  tags                2461 non-null   object
 14  title               2461 non-null   object
 15  urlurl              2461 non-null   object
 16  views               2461

In [10]:
if not joined_exists:
    computing['course'].value_counts()

In [11]:
if not joined_exists:
    economics['course'].value_counts()

In [12]:
if not joined_exists:
    humanities['course'].value_counts()

In [13]:
if not joined_exists:
    math['course'].value_counts()

In [14]:
if not joined_exists:
    science['course'].value_counts()

In [15]:
if not joined_exists:
    khan_dfs = [computing, economics, humanities, math, science]
    khan = pd.concat(khan_dfs, axis=0)
    khan.info()

In [16]:
# if not joined_exists:
# remap the courses to more broad categories: https://stackoverflow.com/a/16476974
labels = ['physics', 'chemistry', 'biology', 'algebra', 'geometry', 'statistics', 'calculus', 'history', 'economics', 'computer']
for lbl in labels:
    # print(lbl)
    for index, row in khan.iterrows():
        if lbl in row['course'].lower():
            row['course'] = lbl
khan['course'].value_counts()

algebra                                         1287
calculus                                         947
chemistry                                        909
history                                          769
biology                                          757
physics                                          743
statistics                                       626
economics                                        502
geometry                                         345
Finance and capital markets                      317
Electrical engineering                           198
Pixar in a Box                                   175
US government and civics                         139
Cosmology and astronomy                           89
AP®︎/College US Government and Politics           87
Trigonometry                                      76
computer                                          75
Differential equations                            70
Storytelling                                  

In [17]:
khan.tail(2)

,course,unit,lesson,video_title,about,transcript,topic,transcript_cleaned,transcript_n_entries,tokens
8259,physics,Review for AP Physics 1 exam,AP Physics 1 free response questions 2015,Question 4: 2015 AP Physics 1 free response,Identical spheres falling from the same height...,- [Voiceover] Two identical\r\nspheres are rel...,Science,"['coherent', 'if we want. F sub G. Or we could...",673,"['-', 'two', 'identical', 'spheres', 'released..."
8260,physics,Review for AP Physics 1 exam,AP Physics 1 free response questions 2015,Question 5: 2015 AP Physics 1 free response,Fundamental frequencies (first harmonics) of s...,- [Voiceover] The figure\r\nabove shows a stri...,Science,"['lets answer each of these. So a', 'the funda...",901,"['-', 'figure', 'shows', 'string', 'one', 'end..."


# Dataset Identification
It is necessary to now identify components of the dataset that can be used for the graphical machine learning. This means identifying Nodes, Edges, Node Features, and Labels. It also includes optionally including edge weights and edge features. For the sake of simplicity I think that I will be forgoing the edge weights and edge features.

I am attempting to basically do topic modeling, but without the keywords and topics that would be customary of Latent Dirchlet Allocation. Instead the goal is to train a Graph ML model using the Khan academy data.

There are two basic routes I could take. I could perform node level prediction by treating each transcript individually and seeing which is the closest match during prediction. The other option would be to perform graph level prediction by storing all of the similarly labeled transcripts together and then using the shape of the graph for comparison. 

- Nodes (Items, People, Locations, Cars, ETC)
- Edges (Connections, Interactions, Similarity, ETC)
    - Levenshtein distance over titles?
    - Number of similar named entities from SpaCy NER?
- Node Features (Attributes)
- Labels (Node-Level, Edge-Level, Graph-Level, etc)
    - I am going to first try node-level prediction as it makes more sense to me. For this I am going to use the 'course' feature in the above pandas dataframe. This will be the target that I try to predict.

In [18]:
if not joined_exists:
    # It takes 21 min to run SpaCy preprocessing over each record in the data
    khan['transcript_cleaned'] = khan['transcript'].progress_apply(lambda x: preprocessor.clean(x, fast=True))
    print(khan['transcript_cleaned'][0])

In [19]:
if not joined_exists:
    # clear up the lists to be unique only.
    khan['tokens'] = khan['transcript'].progress_apply(lambda x: preprocessor.clean(x, tokenize=True, fast=True))
    print(len(khan['tokens'][0]))
    print(khan['tokens'][0])

In [20]:
if not joined_exists:
# if not False:
    khan['transcript_n_entries'] = khan['tokens'].progress_apply(lambda x: len(x))
    khan.head(5)

In [21]:
# khan = khan.drop(columns=["Unnamed: 0.1", "Unnamed: 0"])

In [22]:
khan.head(1)

,course,unit,lesson,video_title,about,transcript,topic,transcript_cleaned,transcript_n_entries,tokens
0,computer,Intro to JS: Drawing & Animation,Intro to programming,What is Programming?,Programming is the process of creating a set o...,"Hi, welcome to programming! If you've never le...",Computing,"['Angry Birds', 'and of course', 'were giving ...",183,"['hi,', 'welcome', 'programming!', 'never', 'l..."


In [23]:
if not joined_exists:
# if not False:
    khan.to_csv("khan_joined.csv", index=False)

In [24]:
# https://www.kaggle.com/code/pavansanagapati/knowledge-graph-nlp-tutorial-bert-spacy-nltk/notebook

"""
    nodes closer to yourself are more important. Take the incoming embeddings and dotproduct them with your own embedding.
    Softmax across all the dot products of the nodes coming in to have it be a probability.
    
    Need to build the corpus of all input words.
    
    Use spacy to get corpus for each doc with stopwords stripped and bigrams / trigrams joined. 
    
    Could use skipgram or bag of words with word2vec to embed words.
"""

'\n    nodes closer to yourself are more important. Take the incoming embeddings and dotproduct them with your own embedding.\n    Softmax across all the dot products of the nodes coming in to have it be a probability.\n    \n    Need to build the corpus of all input words.\n    \n    Use spacy to get corpus for each doc with stopwords stripped and bigrams / trigrams joined. \n    \n    Could use skipgram or bag of words with word2vec to embed words.\n'

In [25]:
khan.head(5)

,course,unit,lesson,video_title,about,transcript,topic,transcript_cleaned,transcript_n_entries,tokens
0,computer,Intro to JS: Drawing & Animation,Intro to programming,What is Programming?,Programming is the process of creating a set o...,"Hi, welcome to programming! If you've never le...",Computing,"['Angry Birds', 'and of course', 'were giving ...",183,"['hi,', 'welcome', 'programming!', 'never', 'l..."
1,computer,Intro to JS: Drawing & Animation,Coloring,The Power of the Docs,Created by Pamela Fox.,Voiceover: Ok so you've\r\nmade a few programs...,Computing,"['because 1 degree to 270 degrees,\r\nthats wh...",542,"['voiceover:', 'ok', 'made', 'programs,', 'mig..."
2,computer,Intro to HTML/CSS: Making webpages,Further learning,HTML validation,Learn how to validate your webpages with the W...,"- [Voiceover] On Khan Academy, we pop up the o...",Computing,['this is a classic thing to forget is that al...,172,"['-', 'khan', 'academy,', 'pop', 'oh', 'noes',..."
3,computer,Intro to SQL: Querying and managing data,SQL basics,Welcome to SQL,SQL is useful for creating and querying relati...,- [Instructor] The world is full of data. Ever...,Computing,['just by mapping user IDs to badge IDs. Thats...,203,"['-', 'world', 'full', 'data.', 'every', 'app'..."
4,computer,Intro to SQL: Querying and managing data,SQL basics,S-Q-L or SEQUEL?,How is it pronounced? Why? Let's discuss...,"At this point, you've probably heard me\r\npro...",Computing,"['At this point', 'by pronouncing SQL the long...",129,"['point,', 'probably', 'heard', 'pronounce', '..."


In [26]:
khan['course'].value_counts()

algebra                                         1287
calculus                                         947
chemistry                                        909
history                                          769
biology                                          757
physics                                          743
statistics                                       626
economics                                        502
geometry                                         345
Finance and capital markets                      317
Electrical engineering                           198
Pixar in a Box                                   175
US government and civics                         139
Cosmology and astronomy                           89
AP®︎/College US Government and Politics           87
Trigonometry                                      76
computer                                          75
Differential equations                            70
Storytelling                                  

In [27]:
# khan['transcript_cleaned'] = khan['transcript']

In [ ]:
corpus = khan['tokens'].sum(axis=0)
if isinstance(corpus, str):
    # means it performed string concatenation so it needs to be cleaned up
    print(corpus[0:100])
    # corpus = str_to_list(corpus)
    print(corpus[0: 25])
    print(len(corpus))
    print(type(corpus))

In [29]:
# convert the corpus to a set since there should be no unique values
corpus = set(corpus)
print(len(corpus))
corpus = list(corpus)
print(corpus[0:25])

380582
['', 'maybe to the same bank,\r\nmaybe a different bank. But it might now have\r\na different interest rate', 'so if we input a into our function then we output -6. f of a is -6. We input b we get three,\r\nwe input c we get -6', 'lets see. 11 times 12 is 121. And then 121 - 1 is going to be', 'maybe,\r\nas a sound wave', 'so thats\r\nthe same thing as-- actually let me just write it this way--\r\nthis is the same thing as 6 to the-- instead of 1/2,\r\nwe can write it as 5/10. Plus 3/5 is the same\r\nthing as 6/10 power', 'and at negative seven', 'which was shorter', 'I learned about computer\r\ngraphics in college because I loved animation. I wasnt crazy about computers', 'thats the Greek letter delta,\r\nshorthand for "change in," well', 'when I multiply them,\r\nequal 64', 'sometimes youll see them in\r\nthese space-filling model', 'if I plot this on a curve', 'when they talk about inflation', 'and so you see these\r\nobligations essentially just growing dramatically. On top 

In [30]:
num_topics = len(khan['course'].value_counts())
print(num_topics)

22


In [31]:
%%time
# LDA Normal
temp = [d.split() for d in corpus]
print(type(temp))
words = corpora.Dictionary(temp)
words.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)
corpus = [words.doc2bow(doc) for doc in temp]

<class 'list'>
CPU times: total: 15.4 s
Wall time: 15.6 s


In [32]:
%%time
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=words,
                                           num_topics=num_topics,
                                           random_state=2,
                                           update_every=1,
                                           passes=15,
                                           alpha='auto')

KeyboardInterrupt: 

In [ ]:
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, words, mds='mmds', R=10)

In [ ]:
vis

In [ ]:
validation.head(5)

In [ ]:
new_text_corpus = words.doc2bow(validation['transcript'][0].split())
print(len(new_text_corpus))
prediction = lda_model.get_document_topics(new_text_corpus)
prediction.sort(key = lambda x: x[1], reverse = True)
print(prediction)
prediction = prediction[0][0]
print("Predicted Topic: %d" % prediction)
lda_model.show_topic(3)